In [ ]:
### test
import numpy as np
import os
import glob
from tensorflow.python.client import device_lib
import tensorflow as tf
# print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import numpy as np
from tqdm import tqdm
import os
import glob  # glob 라이브러리 추가
import pickle
from tensorflow.keras.utils import to_categorical
from datetime import datetime
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(f'C:/Users/oem/Desktop/jhy/array/{start_time}',exist_ok=True)

# dataset 폴더 경로 설정
# PATH = '/content/drive/MyDrive/LAB/Sign_Language_Remaster/code/lstm/dataset'
PATH = r'C:/Users\oem\Desktop\jhy\dataset'

# dataset 폴더 아래의 모든 폴더 목록을 얻기
actions = []  # 변경된 부분
label_mapping = {}  # 변경된 부분

### 액션 리스트 확인
for folder in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, folder)):
        # 각 폴더에 있는 npy 파일들을 확인
        npy_files = glob.glob(os.path.join(PATH, folder, 'seq_*.npy'))
        # 특정 조건에 맞는 npy 파일이 존재하면 actions와 label_mapping에 추가
        if npy_files:
            actions.append(folder)
            label_mapping[folder] = len(actions) - 1
print(len(actions), actions)

# 리스트를 파일로 저장
with open(r'C:\Users\oem\Desktop\jhy\signlanguage\Sign_Language_Remaster\logs\act_list.pkl', 'wb') as f:
    pickle.dump(actions, f)
data = []
labels = []


################################################################################################################################

def create_groups(data_count, group_size, appearances_per_data):
    # 데이터셋 초기화
    datasets = list(range(1, data_count + 1))
   
    # 각 데이터의 사용 횟수를 추적하는 딕셔너리
    usage_count = {dataset: 0 for dataset in datasets}
   
    # 그룹을 저장할 리스트
    groups = []
   
    # 현재 그룹
    current_group = []
   
    # 데이터셋을 순환하면서 그룹 생성
    for _ in range(appearances_per_data):
        for dataset in datasets:
            # 현재 그룹에 데이터 추가
            current_group.append(dataset)
            usage_count[dataset] += 1
           
            # 현재 그룹이 가득 찼거나 모든 데이터가 사용된 경우 그룹 저장 및 초기화
            if len(current_group) == group_size or all(usage_count[dataset] == appearances_per_data for dataset in datasets):
                groups.append(current_group)
                current_group = []
   
    return groups

# 4000개의 데이터셋으로 크기가 500인 그룹을 만들고, 각 데이터가 3번씩 포함되도록 그룹 생성
groups = create_groups(len(actions), 500, 2)

for group_idx, group in enumerate(groups):
    for act_idx in group:
        action = actions[act_idx]
################################################################################################################################
        PATH2 = os.path.join(PATH, action) #/dataset/{action}
        file_path = glob.glob(os.path.join(PATH2, f'seq_*.npy'))[0]
        try:
            loaded_data = np.load(file_path, allow_pickle=True).astype(np.float16)
            data.append(loaded_data)

            label = np.full((loaded_data.shape[0],), label_mapping[action])
            labels.append(label)
        except FileNotFoundError:
            print(f"File not found for action: {action}")

    x_data = np.concatenate(data, axis=0)
    labels = np.concatenate(labels, axis=0)
    y_data = to_categorical(labels, num_classes=len(actions))
    
    np.save(f'C:/Users/oem/Desktop/jhy/array/{start_time}/{group_idx}_XData.npy',x_data)
    np.save(f'C:/Users/oem/Desktop/jhy/array/{start_time}/{group_idx}_YData.npy',y_data)

In [6]:
from datetime import datetime

print(datetime.now().strftime("%Y-%m-%d_%H-%M"))

2024-02-23_20-32-27
